In [8]:
#kütüphanelerin import edilmesi

In [9]:
import pandas as pd
from google.cloud import bigquery, storage
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

In [10]:
#big query ve database bağlantısı

In [11]:
# BigQuery istemcisi
bq_client = bigquery.Client()

# SQL sorgusu – tablo adını kendi projen ve datasetine göre yaz
query = """
SELECT * FROM `bulutprojesi1.akilli_veri_dataset.akilli_veri_tablosu`
"""

# Veriyi pandas DataFrame olarak al
df = bq_client.query(query).to_dataframe()

# İlk 5 satırı göster
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,string_field_32
0,862722,B,6.981,13.43,43.79,143.5,0.11700,0.07568,0.0,0.0,...,19.54,50.41,185.2,0.15840,0.12020,0.0,0.0,0.2932,0.09382,None
1,868999,B,9.738,11.97,61.24,288.5,0.09250,0.04102,0.0,0.0,...,14.10,66.53,342.9,0.12340,0.07204,0.0,0.0,0.3105,0.08151,None
2,872113,B,8.671,14.45,54.42,227.2,0.09138,0.04276,0.0,0.0,...,17.04,58.36,259.2,0.11620,0.07057,0.0,0.0,0.2592,0.07848,None
3,875099,B,9.720,18.22,60.73,288.1,0.06950,0.02344,0.0,0.0,...,20.83,62.25,303.8,0.07117,0.02729,0.0,0.0,0.1909,0.06559,None
4,894047,B,8.597,18.60,54.09,221.2,0.10740,0.05847,0.0,0.0,...,22.44,56.65,240.1,0.13470,0.07767,0.0,0.0,0.3142,0.08116,None


In [12]:
# Özellikler ve etiket (hedef) sütunu ayır
X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]

# Eğitim ve test kümeleri
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Model oluştur ve eğit
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = model.predict(X_test)

# Sonuçları değerlendir
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           B       0.96      0.97      0.97        73
           M       0.95      0.93      0.94        41

    accuracy                           0.96       114
   macro avg       0.95      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



In [14]:
# Modeli yerel olarak kaydet
joblib.dump(model, "model.joblib")

# Cloud Storage istemcisi
storage_client = storage.Client()
bucket_name = "eda-cloud-bucket"  # senin bucket adın
blob = storage_client.bucket(bucket_name).blob("model.joblib")

# Modeli Cloud Storage'a yükle
blob.upload_from_filename("model.joblib")

print("Model başarıyla yüklendi: gs://eda-cloud-bucket/model.joblib")

Model başarıyla yüklendi: gs://eda-cloud-bucket/model.joblib


In [15]:
import joblib
model = joblib.load("model.joblib")
print(model.feature_names_in_)
print(len(model.feature_names_in_))

['id' 'radius_mean' 'texture_mean' 'perimeter_mean' 'area_mean'
 'smoothness_mean' 'compactness_mean' 'concavity_mean'
 'concave points_mean' 'symmetry_mean' 'fractal_dimension_mean'
 'radius_se' 'texture_se' 'perimeter_se' 'area_se' 'smoothness_se'
 'compactness_se' 'concavity_se' 'concave points_se' 'symmetry_se'
 'fractal_dimension_se' 'radius_worst' 'texture_worst' 'perimeter_worst'
 'area_worst' 'smoothness_worst' 'compactness_worst' 'concavity_worst'
 'concave points_worst' 'symmetry_worst' 'fractal_dimension_worst'
 'string_field_32']
32


In [16]:
import numpy as np

# Veri çerçevesini yükle
df = pd.read_csv("data.csv")

# Girdi verisi
input_data = [14.2, 20.5, 95.0, 600.1, 0.1, 0.2, 0.3, 0.4, 0.1, 0.05]  # ilk 10 feature örnek

# Sadece sayısal kolonları al (ilk 10 sütun)
numeric_df = df.select_dtypes(include=[np.number]).iloc[:, :10]

# En yakın satırı bul
distances = numeric_df.apply(lambda row: np.linalg.norm(row - input_data), axis=1)
closest_index = distances.idxmin()

# En yakın satırın gerçek değerini göster
print(df.iloc[closest_index])

id                             8670
diagnosis                         M
radius_mean                   15.46
texture_mean                  19.48
perimeter_mean                101.7
area_mean                     748.9
smoothness_mean              0.1092
compactness_mean             0.1223
concavity_mean               0.1466
concave points_mean         0.08087
symmetry_mean                0.1931
fractal_dimension_mean      0.05796
radius_se                    0.4743
texture_se                   0.7859
perimeter_se                  3.094
area_se                       48.31
smoothness_se               0.00624
compactness_se              0.01484
concavity_se                0.02813
concave points_se           0.01093
symmetry_se                 0.01397
fractal_dimension_se       0.002461
radius_worst                  19.26
texture_worst                  26.0
perimeter_worst               124.9
area_worst                   1156.0
smoothness_worst             0.1546
compactness_worst           

In [17]:
#model test

In [18]:
import requests

url = "https://eda-ml-api-207994772311.us-central1.run.app/predict"

data = {
    "features": [[
        100001, 14.2, 20.5, 95.0, 600.1, 0.1, 0.2, 0.3, 0.2, 0.25, 0.05,
        0.3, 0.4, 0.5, 10.0, 0.01, 0.02, 0.03, 0.04, 0.06, 0.07,
        16.0, 25.0, 120.0, 900.0, 0.15, 0.25, 0.35, 0.45, 0.55, 0.065,
        0.0  # <--- Burayı 'X' yerine 0.0 yaptık
    ]]
}

response = requests.post(url, json=data)
print("Status code:", response.status_code)
print("Prediction:", response.json())

Status code: 200
Prediction: {'prediction': ['M']}
